# 피쳐 추가하여 데이터 만들기

In [1]:
# 모듈 불러오기 및 설정
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

## 취급액 클러스터 추가
1. 사용 데이터: 취급액 합계 기준, 가우지안 스무딩 표준편차 3, KMeans 클러스터링 5.
2. 사용 feature
    - 계절
    - 월
    - 일
    - 요일
    - 방송시간대
    - 노출(분)
    - 상품군
    - 마더코드
    - 상품코드
    - 판매단가
    - 클러스터
    - 클러스터별 시간대별 centroid
3. 제외해야 할 것
    - 2시 방송 데이터
    - 2020년 데이터

In [2]:
# 데이터 경로 설정
RAW_DATA = "./data/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx"
CLUSTER_DATA = "./data/revenue_smoothing3_cluster5.pickle"

In [3]:
# 클러스터 데이터 로드
cluster = pd.read_pickle(CLUSTER_DATA)
cluster.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,cluster,centroid
0,-1.206935,-1.028937,-0.420937,-0.523678,1.454273,-0.718454,0.662448,-0.282473,-0.184177,-0.579907,...,1.299242,0.644134,1.746161,0.223654,-0.232235,-0.153611,-1.590273,-1.590273,4,"[-0.4876779989647162, -0.42678567369229664, -0..."
1,-1.869510,-0.847307,-0.945148,-0.757224,1.546495,-0.125840,0.695929,0.354832,-0.437825,1.564644,...,0.014366,0.799609,0.778627,0.252355,1.453864,0.172290,-1.157775,-1.622847,4,"[-0.4876779989647162, -0.42678567369229664, -0..."
2,-1.600036,-0.447079,-0.577038,-0.651282,-0.004346,-0.399657,-0.318037,0.297276,0.366631,0.986877,...,0.999556,-0.166800,2.826008,1.590362,-0.345687,-0.511546,-1.106934,-1.040089,2,"[-0.7113570503893833, -0.6607207120666089, -0...."
3,-0.963338,-0.762117,-1.219965,-0.349518,0.995123,-0.537472,0.709283,-0.315443,-0.468089,0.628850,...,1.185785,2.055063,1.814503,0.312352,-0.009964,-0.699507,-1.387849,-1.649649,4,"[-0.4876779989647162, -0.42678567369229664, -0..."
4,-1.574757,-1.335667,-0.185116,0.038847,-0.800615,0.407108,0.429081,0.736881,0.256748,-0.588709,...,-0.875403,0.863882,-0.658020,2.353641,0.694420,-0.601883,-1.184539,-0.545846,2,"[-0.7113570503893833, -0.6607207120666089, -0...."


In [4]:
# 날짜 일련번호 - 클러스터 매칭
day_cluster_mapping_dict = {}
for i in cluster.index:
    day_cluster_mapping_dict[i] = cluster['cluster'][i]
print(day_cluster_mapping_dict)

{0: 4, 1: 4, 2: 2, 3: 4, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 1, 10: 2, 11: 0, 12: 4, 13: 2, 14: 2, 15: 2, 16: 2, 17: 1, 18: 0, 19: 4, 20: 2, 21: 4, 22: 2, 23: 4, 24: 1, 25: 4, 26: 4, 27: 2, 28: 1, 29: 1, 30: 3, 31: 1, 32: 3, 33: 4, 34: 2, 35: 2, 36: 2, 37: 2, 38: 1, 39: 4, 40: 4, 41: 2, 42: 1, 43: 2, 44: 2, 45: 2, 46: 4, 47: 4, 48: 2, 49: 2, 50: 2, 51: 1, 52: 1, 53: 1, 54: 4, 55: 2, 56: 1, 57: 1, 58: 1, 59: 2, 60: 4, 61: 1, 62: 3, 63: 1, 64: 4, 65: 3, 66: 1, 67: 0, 68: 3, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 3, 75: 0, 76: 1, 77: 3, 78: 1, 79: 2, 80: 2, 81: 0, 82: 3, 83: 1, 84: 3, 85: 3, 86: 3, 87: 1, 88: 3, 89: 3, 90: 1, 91: 2, 92: 2, 93: 1, 94: 1, 95: 0, 96: 0, 97: 1, 98: 2, 99: 3, 100: 1, 101: 1, 102: 0, 103: 3, 104: 1, 105: 1, 106: 1, 107: 4, 108: 2, 109: 0, 110: 0, 111: 1, 112: 4, 113: 2, 114: 3, 115: 2, 116: 0, 117: 0, 118: 2, 119: 0, 120: 1, 121: 1, 122: 1, 123: 0, 124: 0, 125: 2, 126: 3, 127: 2, 128: 3, 129: 1, 130: 0, 131: 0, 132: 1, 133: 4, 134: 1, 135: 2, 136: 0, 137: 0, 138: 

In [5]:
# 클러스터-centroid 매칭
cluster_centroid_mapping_dict = {}
for i in range(len(cluster)):
    c = cluster['cluster'][i]
    if c in cluster_centroid_mapping_dict:
        continue
    centroid = cluster['centroid'][i]
    temp = {}
    for idx, value in enumerate(centroid):
        temp[idx+6] = value
    cluster_centroid_mapping_dict[c] = temp
    if len(cluster_centroid_mapping_dict) == 5:
        break
print(cluster_centroid_mapping_dict)

{4: {6: -0.4876779989647162, 7: -0.42678567369229664, 8: -0.31897452276397503, 9: -0.18618017735302383, 10: -0.04983226511657912, 11: 0.0754097570199696, 12: 0.18307198796476107, 13: 0.27185624665523933, 14: 0.3414989790188419, 15: 0.3902858589909609, 16: 0.4150142005807905, 17: 0.4120057169056673, 18: 0.37727296925921494, 19: 0.3060707262809029, 20: 0.19396261642620327, 21: 0.04101445303466733, 22: -0.14208036513545486, 23: -0.3303416418793183, 24: -0.48771823648137824, 25: -0.577872630750477}, 2: {6: -0.7113570503893833, 7: -0.6607207120666089, 8: -0.570374484389232, 9: -0.4573381434320238, 10: -0.3377174285132083, 11: -0.22115406240454355, 12: -0.10944633128360548, 13: 0.0006271353692828135, 14: 0.11261635801429026, 15: 0.22652038510141045, 16: 0.3372282463848739, 17: 0.43471683096543895, 18: 0.5049313460871467, 19: 0.5312932481888366, 20: 0.49860825831784905, 21: 0.40007593689259785, 22: 0.24537294487050793, 23: 0.06464022063002908, 24: -0.09660361997406193, 25: -0.1919190783695962

In [16]:
# 제공 실적 데이터 로드
data_raw = pd.read_excel(RAW_DATA, header=1)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    38309 non-null  datetime64[ns]
 1   노출(분)   21525 non-null  float64       
 2   마더코드    38309 non-null  int64         
 3   상품코드    38309 non-null  int64         
 4   상품명     38309 non-null  object        
 5   상품군     38309 non-null  object        
 6   판매단가    38309 non-null  int64         
 7   취급액     35379 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [17]:
# 기본 데이터 조작
data_raw = data_raw[data_raw['상품군']!='무형'] # 무형 상품군 제외
data_raw['노출(분)'] = data_raw['노출(분)'].fillna(method='ffill')
data_raw['취급액'] = data_raw['취급액'].fillna(0)
data_raw = data_raw[data_raw['방송일시'].dt.year == 2019]
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37368 entries, 0 to 38299
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    37368 non-null  datetime64[ns]
 1   노출(분)   37368 non-null  float64       
 2   마더코드    37368 non-null  int64         
 3   상품코드    37368 non-null  int64         
 4   상품명     37368 non-null  object        
 5   상품군     37368 non-null  object        
 6   판매단가    37368 non-null  int64         
 7   취급액     37368 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.6+ MB


In [14]:
# 원본 보존
data = data_raw.copy()

In [15]:
# 피쳐 추가 및 변경
def change_broadcast_hour(x):
    if x == 0:
        return 24
    elif x == 1:
        return 25
    elif x == 2:
        return 26
    else:
        return x

day_mapping_dict = {0:'월요일', 1:'화요일', 2:'수요일', 3:'목요일', 4:'금요일', 5:'토요일', 6:'일요일'}
season_mapping_dict = {1:'겨울', 2:'겨울', 3:'봄', 4:'봄', 5:'봄', 6:'여름', 7:'여름', 8:'여름', 9:'가을', 10:'가을', 11:'가을', 12:'겨울'}

data['월'] = data['방송일시'].dt.month
data['계절'] = data['월'].map(season_mapping_dict)
data['일'] = data['방송일시'].dt.day
data['일_일련번호'] = data['방송일시'].dt.dayofyear-1
data['요일'] = data['방송일시'].dt.dayofweek
data['요일'] = data['요일'].map(day_mapping_dict)
data['방송시간대'] = data['방송일시'].dt.hour.apply(lambda x: change_broadcast_hour(x))
data['클러스터'] = data['일_일련번호'].map(day_cluster_mapping_dict)
data['클러스터_centroid'] = list(zip(data['클러스터'], data['방송시간대'])) # 클러스터, 방송시간대로 컬럼 생성
data['클러스터_centroid'] = data['클러스터_centroid'].map(lambda x: cluster_centroid_mapping_dict.get(x[0]).get(x[1]))
data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,월,계절,일,일_일련번호,요일,방송시간대,클러스터,클러스터_centroid
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,겨울,1,0,화요일,6,4,-0.487678
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,겨울,1,0,화요일,6,4,-0.487678
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,겨울,1,0,화요일,6,4,-0.487678
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,겨울,1,0,화요일,6,4,-0.487678
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,겨울,1,0,화요일,6,4,-0.487678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,2019-12-31 23:20:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,겨울,31,364,화요일,23,3,-0.019946
38296,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,겨울,31,364,화요일,23,3,-0.019946
38297,2019-12-31 23:40:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,겨울,31,364,화요일,23,3,-0.019946
38298,2019-12-31 23:40:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,겨울,31,364,화요일,23,3,-0.019946


In [10]:
# 클러스터 centroid 없는 데이터 삭제: 새벽 2시
print(data.isnull().sum())
df = data.dropna() # 원본 보존 및 결측치 제거
df

방송일시              0
노출(분)             0
마더코드              0
상품코드              0
상품명               0
상품군               0
판매단가              0
취급액               0
월                 0
계절                0
일                 0
일_일련번호            0
요일                0
방송시간대             0
클러스터              0
클러스터_centroid    70
dtype: int64


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,월,계절,일,일_일련번호,요일,방송시간대,클러스터,클러스터_centroid
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,겨울,1,0,화요일,6,4,-0.487678
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,겨울,1,0,화요일,6,4,-0.487678
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,겨울,1,0,화요일,6,4,-0.487678
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,겨울,1,0,화요일,6,4,-0.487678
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,겨울,1,0,화요일,6,4,-0.487678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,2019-12-31 23:20:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,겨울,31,364,화요일,23,3,-0.019946
38296,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,겨울,31,364,화요일,23,3,-0.019946
38297,2019-12-31 23:40:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,겨울,31,364,화요일,23,3,-0.019946
38298,2019-12-31 23:40:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,겨울,31,364,화요일,23,3,-0.019946


In [11]:
# 인덱스 정렬
df = df.reset_index(drop=True)
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,월,계절,일,일_일련번호,요일,방송시간대,클러스터,클러스터_centroid
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,겨울,1,0,화요일,6,4,-0.487678
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,겨울,1,0,화요일,6,4,-0.487678
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,겨울,1,0,화요일,6,4,-0.487678
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,겨울,1,0,화요일,6,4,-0.487678
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,겨울,1,0,화요일,6,4,-0.487678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37293,2019-12-31 23:20:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,겨울,31,364,화요일,23,3,-0.019946
37294,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,겨울,31,364,화요일,23,3,-0.019946
37295,2019-12-31 23:40:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,겨울,31,364,화요일,23,3,-0.019946
37296,2019-12-31 23:40:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,겨울,31,364,화요일,23,3,-0.019946


In [13]:
# 데이터 저장
df.to_csv("./data/rawdata_gau3_revClu5.csv", index=False, encoding='utf-8-sig')